In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'

import os
os.environ["WANDB_DISABLED"] = "true"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
! nvcc --version
!nvidia-smi -L
device

In [ ]:
!pip install -U transformers
#!pip install transformers==4.28.0
!pip install transformers==4.45.2
!pip install datasets
!pip install snscrape
!pip install --upgrade accelerate
!pip install datasets evaluate
!pip install accelerate -U

from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer
import numpy as np
import random
import csv
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import datasets
from datasets import Dataset, DatasetDict
import evaluate
from sklearn.metrics import classification_report, f1_score

import sys, os
import torch
import json
import re


In [ ]:
# preprocess functions

def remove_url(text):
    return ' '.join(re.sub("(\w+:\/\/\S+)"," ", text).split())

def remove_username(text):
    return ' '.join(re.sub("([@][A-Za-z0-9_]+)"," ", text).split())

def remove_at_mark(text):
    return re.sub(r'[@]', ' ', text)

def remove_tag_mark(text):
    return re.sub(r'[#]', ' ', text)

def replace_hashtags_with_segments (text):
   regex = "#(\w+)"
   text_split = text.split()
   sent= ''
   strip_word =' '
   for word in text_split:
     hashtag= re.findall(regex, word)
     #print(hashtag)
     #print(strip_word)

     if (len(hashtag)) == 0 :
        sent = sent + ' ' + word
     if (len(hashtag))!=0:
        strip_word = word.replace(('#'+hashtag[0]), ' ')
        #print (hashtag[0])
        #print(strip_word)
        try :
           key = '#'+ hashtag[0]
           segmented = segmented_dict [key]
           if type(segmented) == str:
                    #segmented = re.sub(r'[|]', ' ', segmented)
                    #segmented = re.sub(r'[,]', ' ', segmented)
                    sent= sent + ' ' + segmented
           if type(segmented) == list:
                    for k in segmented:
                      sent = sent +' ' + k
        except:
           sent = sent + ' ' +  hashtag[0]
   return sent + ' ' + strip_word

def control_tag_mark(text):
     s = ' '
     for word in text.split():
            word = re.sub(r'[#]', ' #', word)
            s = s + ' ' + word
     return s

def remove_punctuation_marks(text):
   punc = '''!()-[]{};:'"\,<>./?$%^&*~'''
   for i in text:
       if i in punc:
         text = text.replace(i, " ")
   return text



def remove_underline(text):
  punc = '''_'''
  for i in text:
       if i in punc:
         text = text.replace(i, "")
  return text

def remove_hashtag(text):
    return ' '.join(re.sub("([#][A-Za-z0-9_]+)"," ", text).split())

def remove_turkish_hashtag(text):
    return ' '.join(re.sub("([#][\w+_]+)"," ", text).split())

def find_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return re.findall(emoji_pattern, text) # no emoji

def remove_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return (emoji_pattern.sub(r'', text)) # no emoji

def remove_arabic_hashtags(text):
   return ' '.join(re.sub('([#][\u0600-\u06ff_]+)', " ", text).split())

def remove_emoji_duplicate (text):
      emoji =[]
      rest = ''
      main_text = []
      for w in text :
         x = find_emoji(w)
         if len (x) != 0 :
            emoji.append (x[0])
      #print(emoji)
      rest = remove_emoji(text)
      emoji = list(set(emoji))
      for e in emoji :
        try:
          rest = rest + " " + e
        except:
          rest = rest
      main_text.append(rest)
      return main_text[0]

def lower_case(text):
    text = text.replace("I", "ı")
    text = text.replace("İ", "i")
    return text.lower()

In [ ]:
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(0)

In [ ]:
# Define data path
# Please replace the addresses with the address of dataset on your local drive

path_turkish_data     =  '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/data/df_turkish_cats_targets_degrees_final_with_translated_all.csv'
path_arabic_data      =  '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/data/df_arabic_cats_targets_degrees_final.csv'
path_synthetic_data   =  '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/data/synthetic_data_all_new.xlsx'

path_all_emoji =         '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/data/all_emoji_dict.json'
path_hashtag_segment  =  '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/data/all_hashtag_segmented_tr.json'

In [ ]:
# Laod emoji_dict and hashtag_dict

f = open(path_all_emoji)
data = json.load(f)
emoji_list = data['emojis']

f = open(path_hashtag_segment)
segmented_dict = json.load(f)

In [ ]:
# Select which preprocess steps must be apply on data

def pre_process_tweets(text_list):
   cleaned_text =[]
   for s in text_list:
       #s = lower_case(s)  #Umut
       s = s.lower()
       s = remove_username(s)
       s = remove_at_mark(s)
       s = remove_url(s)
       s = remove_tag_mark(s)
       #s = control_tag_mark(s)  #
       s = remove_punctuation_marks(s)
       s = remove_underline(s)  #
       #s = replace_hashtags_with_segments(s) #
       #s = remove_tag_mark(s)
       cleaned_text.append(s)
   return cleaned_text

In [ ]:
#Load synthetic data

df_syn = pd.read_excel(path_synthetic_data)
df_syn = df_syn.iloc[:,[4, 10, 9,]]

print(df_syn['DegreeMean'].value_counts())
df_syn= df_syn.sample(frac = 1)

df_syn

In [ ]:
# Load Turkish data

df_turkish = pd.read_csv(path_turkish_data)
df_turkish = df_turkish.iloc[:,[2, 37,38, 39]]

df_turkish

In [ ]:
#Load Arabic data

df_arabic = pd.read_csv(path_arabic_data)
df_arabic = df_arabic.iloc[:,[2,37,38]]

df_arabic

In [ ]:
# select number of classes
# select k-fold for spliting train and validation data

num_class = 1
k_fold = 11


In [ ]:
# Select strength column
# Define the name and the address of trained model on HuggingFace

if num_class == 1:
     df_turkish.dropna(subset=['DegreeMean'], inplace=True)
     df_arabic.dropna(subset=['DegreeMean'], inplace=True)
     hub_id = 'HrantDinkFoundation/'
     model_id = 'hs-degree-prediction'
     column_select = 'DegreeMean'

In [ ]:
# select which data used in training

# 'only-tr' : use only Turkish data
# 'only-ar' : use only arabic data
# 'tr-syn'  : use Turkish and synthetic data
# 'ar-syn'  : use arabic and synthetic data
# 'ar-translated-syn'  : use arabic, synthetic and the translated column of Turkish data

mode_train = ['only-tr', 'only-ar', 'tr-syn' , 'ar-syn', 'ar-translated-syn']
mode = mode_train [2]

In [ ]:
if mode =='only-tr':
    model_lang = 'tr'
    model_name = 'dbmdz/bert-base-turkish-uncased'
    df_train = df_turkish[(df_turkish.Div=='train')]
    df_test  = df_turkish[(df_turkish.Div=='test')]

if mode =='only-ar' :
     model_lang = 'ar'
     model_name = 'aubmindlab/bert-base-arabert'
     df_train = df_arabic[(df_arabic.Div=='train')]
     df_test  = df_arabic[(df_arabic.Div=='test')]


if mode == 'tr-syn':
    model_lang = 'tr'
    hub_id = hub_id + 'turkish-' + model_id
    model_name = 'dbmdz/bert-base-turkish-uncased'
    df_train = df_turkish[(df_turkish.Div=='train')]
    df_test  = df_turkish[(df_turkish.Div=='test')]

    df_train = pd.concat([df_train, df_syn], ignore_index=True)


if mode == 'ar-syn':
    model_lang = 'ar'
    hub_id = hub_id + 'arabic-' + model_id
    model_name = 'aubmindlab/bert-base-arabert'
    df_train = df_arabic[(df_arabic.Div=='train')]
    df_test  = df_arabic[(df_arabic.Div=='test')]

    df_syn = df_syn.drop (columns=['Text'])
    df_syn.rename(columns={'translated_text_arabic': 'Text'}, inplace=True)
    df_train = pd.concat([df_train, df_syn], ignore_index=True)


if mode == 'ar-translated-syn':
    model_lang = 'ar'
    model_name = 'aubmindlab/bert-base-arabert'
    hub_id = hub_id + 'arabic-' + model_id

    df_turkish = df_turkish.drop (columns=['Text'])
    df_turkish.rename(columns={'translated_text_arabic': 'Text'}, inplace=True)

    df_train_arabic = df_arabic[(df_arabic.Div=='train')]
    df_test  = df_arabic[(df_arabic.Div=='test')]

    df_syn = df_syn.drop (columns=['Text'])
    df_syn.rename(columns={'translated_text_arabic': 'Text'}, inplace=True)

    df_train = pd.concat([df_train_arabic, df_turkish, df_syn], ignore_index=True)


In [ ]:
df_train


In [ ]:
df_test

In [ ]:
# preprocess train data

import ast

text = df_train['Text'].tolist()
label = df_train[column_select].tolist()

text = pre_process_tweets(text)
train_data = pd.DataFrame(list(zip(text, label)), columns =['text', 'label'])

train_data= train_data.astype({'label':'float'})

train_data


In [ ]:
# preprocess test data

text = df_test['Text'].tolist()
label = df_test[column_select].tolist()

text = pre_process_tweets(text)
test_data = pd.DataFrame(list(zip(text, label)), columns =['text', 'label'])
test_data= test_data.astype({'label':'float'})


test_data


In [ ]:
# Laod BERT model

from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import ElectraForSequenceClassification,  ElectraTokenizerFast
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer, RobertaTokenizerFast, RobertaForSequenceClassification
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments, GPT2Config

pretrained_model_name = model_name
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name, do_lower_case=True, force_download=True)
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name, num_labels = num_class, force_download=True ).to(device)   # id2label=id2label, label2id=label2id
model.resize_token_embeddings(len(tokenizer))
for item in emoji_list :
        tokenizer.add_tokens(item)
model.resize_token_embeddings(len(tokenizer))
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
def preprocess_function(examples):
    preprocessed = examples["text"]
    return tokenizer(preprocessed, truncation=True)

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics(eval_pred):

    metric4 = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]
    macro_f1 = (f1_score(labels, predictions, average ="macro"))

    return {"accuracy":accuracy, "macro_f1":macro_f1}


def compute_metrics(eval_pred):
    metric1 = evaluate.load("mse")
    logits, labels = eval_pred
    mse = metric1.compute(predictions=logits, references=labels)["mse"]
    return {"mse": mse}



def compute_metrics(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)

    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    single_squared_errors = ((logits - labels).flatten()**2).tolist()

    # Compute accuracy
    # Based on the fact that the rounded score = true score only if |single_squared_errors| < 0.5
    accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)

    return {"mse": mse, "mae": mae, "r2": r2, "accuracy": accuracy}

In [ ]:
skf = StratifiedKFold(n_splits= k_fold, random_state = 31, shuffle=True)

In [ ]:
from torch import nn
from transformers import Trainer

class CustomTrainer(Trainer):
    def save_model(self, output_dir=None, _internal_call=False):
        if output_dir is None:
            output_dir = self.args.output_dir

        self.model = self.model.to('cuda')

        for param in self.model.parameters():
            if not param.is_contiguous():
                param.data = param.data.contiguous()

        super().save_model(output_dir, _internal_call)



class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss_fct = nn.MSELoss()
        loss = loss_fct(logits.squeeze(), labels.squeeze())
        return (loss, outputs) if return_outputs else loss

In [ ]:
# define training and validation data
val_ratio = 0.09

inds = np.random.permutation(train_data.shape[0])
ind_split = int(train_data.shape[0]*val_ratio)

val_data = train_data.loc[inds[:ind_split]].sort_index().copy()
train_data2 = train_data.loc[inds[ind_split:]].sort_index().copy()

assert val_data.shape[0] + train_data2.shape[0] == train_data.shape[0]
train_data2.head(20)


In [ ]:
# Let's start training model

val = Dataset.from_pandas(val_data.set_index("text"))
train = Dataset.from_pandas(train_data2.set_index("text"))

dataset_dict = DatasetDict({"val": val, "train": train})
preprocessed_dataset_dict = dataset_dict.map(preprocess_function, batched=True)

training_args = TrainingArguments(
        "Hate Speech Strength Prediction",
        learning_rate=5e-6,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=20,
        num_train_epochs=10,
        weight_decay=0.01,
        evaluation_strategy="steps",
        metric_for_best_model = 'accuracy',
        logging_steps = 100,
        load_best_model_at_end=True,
        greater_is_better=True,
        #hub_strategy="end",           # These parameter used when you want saved mmodels directly to HuggingFace,
        #push_to_hub=True,             # If you want to save model on huggingFace you must select "True"
        #hub_model_id= hub_id,         # Write your model address on HuggingFace
        #hub_private_repo= False,      # Determine whether the model on Hugging Face should be private or public.
        #hub_token= "...........",     # To save model into HuggingFace you need "hub_token" which can take from your own HuggingFace account
)

trainer = RegressionTrainer(
        model = model,
        args = training_args,
        train_dataset=preprocessed_dataset_dict["train"],
        eval_dataset=preprocessed_dataset_dict["val"],
        tokenizer = tokenizer,
        data_collator = data_collator,
        compute_metrics = compute_metrics,

)
trainer.train()

#trainer.save_model()       # If you want save your trained model on HuggingFace

In [ ]:
# Let's start testing model

test_ds = Dataset.from_pandas(test_data).map(preprocess_function, batched=True)
test_ds

In [ ]:
predictions = trainer.predict(test_ds).predictions
predictions

In [ ]:
test_data['Predicted-HS-Severity'] = predictions.reshape(-1)
test_data

In [ ]:
test_data.loc[test_data['Predicted-HS-Severity'] < 0, 'Predicted-HS-Severity'] = 0
test_data.loc[test_data['Predicted-HS-Severity'] < 0]

test_data['Rounded-HS-Severity'] = test_data['Predicted-HS-Severity'].round()

test_data

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy.stats import pearsonr, spearmanr


MSE_simple =  mean_squared_error(test_data['label'].tolist(), test_data['Predicted-HS-Severity'].tolist())
MSE_rounded = mean_squared_error(test_data['label'].tolist(), test_data['Rounded-HS-Severity'].tolist())


RMSE_simple = sqrt(mean_squared_error(test_data['label'].tolist(), test_data['Predicted-HS-Severity'].tolist()))
RMSE_rounded = sqrt(mean_squared_error(test_data['label'].tolist(), test_data['Rounded-HS-Severity'].tolist()))

correlation = test_data['label'].corr(test_data['Predicted-HS-Severity'])


print("MSE_simple:", MSE_simple )
print("MSE_rounded:", MSE_rounded )
print(" ")
print("RMSE_simple:", RMSE_simple )
print("RMSE_rounded:", RMSE_rounded )
print(" ")
print("Pearson correlation coefficient:", correlation)

In [ ]:
# Save trained model on local drive
# Please replace the pathes with your local address path


if model_lang == 'tr':
          path_save_model = '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/trained_models/hs-tr-strength/'

if model_lang == 'ar':
          path_save_model = '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/trained_models/hs-ar-strength/'


trainer.save_model(path_save_model)

In [ ]:
# Load trained model from local drive

from transformers import pipeline

path_trained_model = path_save_model
model_pipline = pipeline(task = "text-classification", model = path_trained_model, function_to_apply="none" , device="cuda")


In [ ]:
# predict the strength of example teewts using trained model

def predict_strength_tweet(review):
        prediction = []
        prediction = (model_pipline (review))
        prediction = prediction[0]["score"]
        return prediction

🔴 ▶ **Disclaimer:**

This study (project) includes examples of offensive language, hate speech, and profanity due to the nature of the research. These examples do not represent the authors’ views (or developers' views). The goal of this work is to contribute to the detection and prevention of harmful content and violence directed toward refugees, LGBTI+ individuals, religious groups, nationalities, and other vulnerable communities.

In [ ]:
AR_TWEETS_EXAMPLES = [
    "كل اللاجئين لصوص",
    "ارحلوا من بلدنا",
    "مكان المرأة هو المطبخ",
    "اليونان يجب أن تعرف حدودها، وإلا قد نأتي فجأة في ليلة",
    "لا ينبغي لغير المسلمين أن يصبحوا نوابًا.",
    "يجب ألا يُسمح للاجئين بالتصويت.",
    "يجب ألا يستفيد اللاجئون من التأمين الصحي.",
    "التعليم باللغة الكردية غير مقبول.",
    "للاجئين أيضًا الحق في الحياة.",
    "يتم اتخاذ تدابير شاملة ضمن الإطار القانوني لمنع أنشطة التنظيمات الإرهابية.",
    "تم الإعلان عن تواريخ الفعاليات الثقافية التي ستُقام هذا العام في إسرائيل.",
    "تفتتح رائدات الأعمال صالونات تجميل تستهدف العملاء من المسلمين وغير المسلمين.",
    "تُجرى الاستعدادات في الأسواق في سوريا قبل الأعياد الإسلامية والمسيحية.",
    "هل كون بعض الكلاب مسعورة أو عدوانية يبرر قتل جميع الكلاب؟ اسحبوا القانون.",
     "يجب على النساء دعم بعضهن البعض",
    "اشتريت تفاحًا من السوق.",
    "طعام السوريين لذيذ جدًا.",
    "الطعام السوري شهي للغاية.",
    "أنا أحب السوريين كثيرًا، إنهم أشخاص لطيفون جدًا.",
    "السنغافوريون يعملون بجد.",
    "أنا أحب الأمريكيين كثيرًا",
    "أنا لا أحب الملحدين على الإطلاق",
    "المسيحيين الأرثوذكس",
    "سنغافورة بلد جميل جدًا"   ,
    "ذهبت إلى المستشفى أمس، ثم تناولت وجبة طعام لذيذة، ثم عدت إلى المنزل، ولعبت مع ابنتي، وشاهدت مسلسلًا أمريكيًا",
    "أنا أحب الحيوانات كثيرًا"
]

TR_TWEETS_EXAMPLES = [
    "mültecilerin hepsi hırsızdır",
    "defolun bizim ülkemizden",
    "kadının yeri mutfaktır",
    "Yunanistan haddini bilecek yoksa bir gece ansizin gelebiliriz",
    "Gayrimüslimler milletvekili olmasın.",
    "Mülteciler oy kullanmasın.",
    "Mültecilere oy hakki verilmemeli",
    "Mülteciler sgk'dan yararlanmasın.",
    "Kürtçe eğitim kabul edilemez.",
    "Mültecilerin de yaşamaya hakkı var.",
    "Terör örgütlerinin faaliyetlerini önlemek için hukuki çerçevede kapsamlı önlemler alınmaktadır.",
    "İsrail'de bu yıl düzenlenecek kültürel etkinliklerin tarihleri açıklandı",
    "Kadın girişimciler, hem Müslüman hem de diğer topluluklardan müşterilere hitap eden güzellik salonları açıyor.",
    "Suriye’deki pazarlarda Müslüman ve Hristiyan bayramları öncesinde hazırlıklar yapılıyor.",
    "Bazı köpeklerin kuduz ya da saldırgan olması, tüm köpeklerin öldürülmesi için haklı bir sebep midir? Yasayı geri çek.",
    "Kadınlar birbirlerine destek olmalı.",
    "Pazardan elma aldim.",
    "Suriyelilerin yemekleri çok güzel.",
    "Suriye yemekleri çok lezzetli.",
    "Süriyelileri çok seviyorum, çok sempatik insanlar.",
    "Singapurlular çok çalışkan",
    "Amerikalıları çok Seviyorum",
    "Ateistleri hiç sevmiyorum",
    "Ortodoksalar hrıstiyanlar",
    "Singapur çok güzel bir ülke dir",
    "Dün hastaneye gittim sonra da bir güzel yemek yedim, eve döndüm, kızımla oyun oynadım, Amerikan dizi izledim",
    "hayvanlari çok seviyorum"
]


if model_lang == 'ar':
      TWEETS_EXAMPLES = AR_TWEETS_EXAMPLES

if model_lang == 'tr':
      TWEETS_EXAMPLES = TR_TWEETS_EXAMPLES

for tweet in TWEETS_EXAMPLES:
    strength = predict_strength_tweet(remove_punctuation_marks(tweet.lower()))
    strength = round (strength)
    print(f"Tweet:    {tweet}")
    print(f"Strength: {strength}")
    print("-" * 60)
